In [2]:
import pandas as pd
from dotenv import load_dotenv
# from intensity import extract_complete_trajectory_features
from gender_classifier import classify_gender
import librosa
data_dir = "enhanced_audio/"
raw_dir = "../../../MELD.Raw"

Data_processing

In [9]:
data_df = pd.read_csv(raw_dir+"/train/train_sent_emo.csv")
data_df
# train_data_dir = data_dir+"/train/train_splits/"

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tr...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
9984,10474,You or me?,Chandler,neutral,neutral,1038,13,2,3,"00:00:48,173","00:00:50,799"
9985,10475,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,neutral,neutral,1038,14,2,3,"00:00:51,009","00:00:53,594"
9986,10476,"You guys are messing with me, right?",Joey,surprise,positive,1038,15,2,3,"00:01:00,518","00:01:03,520"
9987,10477,Yeah.,All,neutral,neutral,1038,16,2,3,"00:01:05,398","00:01:07,274"


In [11]:
data_df.drop(columns=['Utterance_ID', 'Dialogue_ID','Sentiment', 'Sr No.', 'Utterance', 'Speaker', 'Episode', 'Season', 'StartTime', 'EndTime'], inplace=True)
data_df

,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID
0,1,neutral,neutral,0,0
1,2,neutral,neutral,0,1
2,3,neutral,neutral,0,2
3,4,neutral,neutral,0,3
4,5,surprise,positive,0,4
...,...,...,...,...,...
9984,10474,neutral,neutral,1038,13
9985,10475,neutral,neutral,1038,14
9986,10476,surprise,positive,1038,15
9987,10477,neutral,neutral,1038,16


In [13]:
data_df =data_df[data_df['Emotion']=="joy"]
data_df

,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID
23,26,joy,positive,2,2
31,34,joy,positive,2,10
33,36,joy,positive,2,12
44,48,joy,positive,4,1
51,55,joy,positive,4,8
...,...,...,...,...,...
9938,10428,joy,positive,1034,1
9946,10436,joy,positive,1035,6
9947,10437,joy,positive,1036,0
9972,10462,joy,positive,1038,1


In [17]:
def data_formatter(utt_id, dia_id):
    return f"dia{dia_id}_utt{utt_id}_enhanced.wav"

# Create Video_ID column using vectorized operations instead of loop
data_df['Video_ID'] = data_df.apply(lambda row: data_formatter(str(row['Utterance_ID']), str(row['Dialogue_ID'])), axis=1)
data_df.drop(columns=['Utterance_ID', 'Dialogue_ID','Sentiment', 'Sr No.'], inplace=True)
data_df


/var/folders/lq/789xtspj42b7fzl0zns5n46w0000gn/T/ipykernel_74218/2959465266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['Video_ID'] = data_df.apply(lambda row: data_formatter(str(row['Utterance_ID']), str(row['Dialogue_ID'])), axis=1)


,Sr No.,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Video_ID
23,26,joy,positive,2,2,dia2_utt2_enhanced.mp4
31,34,joy,positive,2,10,dia2_utt10_enhanced.mp4
33,36,joy,positive,2,12,dia2_utt12_enhanced.mp4
44,48,joy,positive,4,1,dia4_utt1_enhanced.mp4
51,55,joy,positive,4,8,dia4_utt8_enhanced.mp4
...,...,...,...,...,...,...
9938,10428,joy,positive,1034,1,dia1034_utt1_enhanced.mp4
9946,10436,joy,positive,1035,6,dia1035_utt6_enhanced.mp4
9947,10437,joy,positive,1036,0,dia1036_utt0_enhanced.mp4
9972,10462,joy,positive,1038,1,dia1038_utt1_enhanced.mp4


In [45]:
# Create a new column for Gender using loc to avoid SettingWithCopyWarning
data_df.loc[:, 'Gender'] = None

# Process each row using loc to avoid SettingWithCopyWarning
for row in range(len(data_df)):
    video_path = "enhanced_audio/" + data_df.iloc[row]['Video_ID']
    gender = classify_gender(video_path)
    data_df.loc[row, 'Gender'] = gender

Classification result: male
Classification result: male
Classification result: male
Classification result: female
Classification result: female
Classification result: male
Classification result: female
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: female
Classification result: female
Classification result: male
Classification result: female
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: male
Classification result: female
Classification result: female
Classification result: female
Classification result: male
Classification result: male
Classification result: male
Classification result: female


In [47]:
data_df.to_csv("joy_data.csv", index=False)

In [3]:
data_df = pd.read_csv("joy_data.csv")

In [4]:
data_df

,Emotion,Video_ID,Gender
0,joy,dia2_utt2_enhanced.wav,male
1,joy,dia2_utt10_enhanced.wav,male
2,joy,dia2_utt12_enhanced.wav,male
3,joy,dia4_utt1_enhanced.wav,female
4,joy,dia4_utt8_enhanced.wav,female
...,...,...,...
1738,joy,dia1034_utt1_enhanced.wav,female
1739,joy,dia1035_utt6_enhanced.wav,male
1740,joy,dia1036_utt0_enhanced.wav,male
1741,joy,dia1038_utt1_enhanced.wav,female


In [5]:
import librosa
import numpy as np
import scipy.stats
import scipy.signal  

def compute_linear_trend(feature_sequence):
    """
    Calculate the slope of the linear trend in a feature sequence.
    Positive values indicate rising trend, negative values indicate falling trend.
    
    Args:
        feature_sequence: Array of feature values over time
        
    Returns:
        Slope of the best-fit line

    Reference:

    Schuller, B., Steidl, S., Batliner, A., Burkhardt, F., Devillers, L., Müller, C., & Narayanan, S. (2013). 
        Paralinguistics in speech and language—State-of-the-art and the challenge. Computer Speech & Language, 27(1), 4-39.
        Shows linear regression coefficients of energy contours as key predictors of emotion intensity.

    Batliner, A., Steidl, S., Schuller, B., Seppi, D., Vogt, T., Wagner, J., ... & Amir, N. (2011).
        Whodunnit-searching for the most important feature types signalling emotion-related user states in speech.
        Computer Speech & Language, 25(1), 4-28. Validates linear trend features for emotion intensity prediction.

    """
    # Create time indices (normalized to [0,1])
    feature_sequence = np.array(feature_sequence)  
    n_frames = len(feature_sequence)
    
    # Handle edge cases
    if n_frames < 2:
        return 0.0
    
    # Check if array contains only identical values
    if np.all(feature_sequence == feature_sequence[0]):
        return 0.0
    
    time_indices = np.linspace(0, 1, n_frames)
    
    # Compute linear regression
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
        time_indices, feature_sequence
    )
    
    # Normalize by feature range to handle different scales
    feature_range = np.max(feature_sequence) - np.min(feature_sequence)
    if feature_range > 0:
        normalized_slope = slope / feature_range
    else:
        normalized_slope = 0.0
    
    return normalized_slope

def compute_feature_dynamism(feature_sequence):
    """
    Calculate the dynamism (variability in rate of change) of a feature.
    Higher values indicate more dynamic expression.
    
    Args:
        feature_sequence: Array of feature values over time
        
    Returns:
        Dynamism score

    References:
    
    Schuller, B., Batliner, A., Steidl, S., & Seppi, D. (2011). 
        Recognising realistic emotions and affect in speech: State of the art and lessons learnt from the first challenge.Speech Communication, 53(9-10), 1062-1087.
        Demonstrates that first and second derivatives of prosodic features significantly improve emotion intensity recognition.

    Eyben, F., Scherer, K. R., Schuller, B. W., Sundberg, J., André, E., Busso, C., ... & Truong, K. P. (2016). 
        The Geneva minimalistic acoustic parameter set (GeMAPS) for voice research and affective computing. IEEE Transactions on Affective Computing, 7(2), 190-202. 
        Defines dynamism features as key performance indicators for emotion intensity detection.
    """
    feature_sequence = np.array(feature_sequence)  
    
    # Handle edge cases
    if len(feature_sequence) < 3:  # Need at least 3 points for second derivative
        print ("not enough len")
        return 0.0
    
    # Compute first derivative (rate of change)
    first_derivative = np.diff(feature_sequence)
    
    # Compute second derivative (acceleration)
    second_derivative = np.diff(first_derivative)
    
    # Calculate metrics
    first_derivative_mean = np.mean(np.abs(first_derivative))
    first_derivative_std = np.std(first_derivative)
    second_derivative_mean = np.mean(np.abs(second_derivative))
    
    # Combine into dynamism measure (normalized to feature range)
    feature_range = np.max(feature_sequence) - np.min(feature_sequence)
    if feature_range > 0:
        dynamism = (0.5 * first_derivative_std + 
                   0.3 * first_derivative_mean + 
                   0.2 * second_derivative_mean) / feature_range
    else:
        dynamism = 0.0
    
    return dynamism

def detect_bursts(feature_sequence, threshold=1.5, min_duration=3):
    """
    Detect burst patterns in a feature sequence.
    
    Args:
        feature_sequence: Array of feature values over time
        threshold: Threshold for burst detection (multiplier over local mean)
        min_duration: Minimum frames to be considered a burst
        
    Returns:
        List of (start_index, end_index, magnitude) for each burst
    """
    feature_sequence = np.array(feature_sequence)  
    
    # Handle empty array
    if len(feature_sequence) == 0:
        return []
    
    # Normalize feature sequence
    feature_mean = np.mean(feature_sequence)
    feature_std = np.std(feature_sequence)
    if feature_std > 0:
        normalized_sequence = (feature_sequence - feature_mean) / feature_std
    else:
        return []
    
    # Detect regions above threshold
    above_threshold = normalized_sequence > threshold
    
    # Find continuous regions (bursts)
    bursts = []
    in_burst = False
    burst_start = 0
    
    for i, is_above in enumerate(above_threshold):
        if is_above and not in_burst:
            # Start of new burst
            in_burst = True
            burst_start = i
        elif not is_above and in_burst:
            # End of burst
            burst_end = i
            burst_duration = burst_end - burst_start
            
            if burst_duration >= min_duration:
                # Calculate burst magnitude
                burst_magnitude = np.mean(normalized_sequence[burst_start:burst_end])
                bursts.append((burst_start, burst_end, burst_magnitude))
            
            in_burst = False
    
    # Handle case where sequence ends during a burst
    if in_burst:
        burst_end = len(above_threshold)
        burst_duration = burst_end - burst_start
        
        if burst_duration >= min_duration:
            burst_magnitude = np.mean(normalized_sequence[burst_start:burst_end])
            bursts.append((burst_start, burst_end, burst_magnitude))
    
    return bursts

def compute_burst_features(feature_sequence, threshold=1.5, min_duration=3):
    """
    Compute features based on burst patterns.
    
    Args:
        feature_sequence: Array of feature values over time
        threshold: Threshold for burst detection
        min_duration: Minimum frames for a burst
        
    Returns:
        Dictionary of burst-related features
    """
    feature_sequence = np.array(feature_sequence)  
    
    bursts = detect_bursts(feature_sequence, threshold, min_duration)
    
    n_frames = len(feature_sequence)
    if n_frames == 0:
        return {
            'burst_count': 0,
            'burst_rate': 0,
            'mean_burst_magnitude': 0,
            'max_burst_magnitude': 0,
            'burst_coverage': 0,
            'first_burst_position': 0
        }
    
    # Compute burst features
    burst_count = len(bursts)
    burst_rate = burst_count / n_frames
    
    if burst_count > 0:
        burst_magnitudes = [b[2] for b in bursts]
        burst_durations = [b[1] - b[0] for b in bursts]
        total_burst_frames = sum(burst_durations)
        
        mean_burst_magnitude = np.mean(burst_magnitudes)
        max_burst_magnitude = np.max(burst_magnitudes)
        burst_coverage = total_burst_frames / n_frames
        first_burst_position = bursts[0][0] / n_frames if bursts else 0
    else:
        mean_burst_magnitude = 0
        max_burst_magnitude = 0
        burst_coverage = 0
        first_burst_position = 0
    
    return {
        'burst_count': burst_count,
        'burst_rate': burst_rate,
        'mean_burst_magnitude': mean_burst_magnitude,
        'max_burst_magnitude': max_burst_magnitude,
        'burst_coverage': burst_coverage,
        'first_burst_position': first_burst_position
    }

def analyze_peaks(feature_sequence, distance=5, prominence=0.1, width=None):
    """
    Perform comprehensive peak analysis on a feature sequence.
    
    Args:
        feature_sequence: Array of feature values over time
        distance: Minimum samples between peaks
        prominence: Minimum peak prominence
        width: Minimum peak width
        
    Returns:
        Dictionary of peak-related features
    """
    feature_sequence = np.array(feature_sequence)  
    
    # Handle empty array
    n_frames = len(feature_sequence)
    if n_frames == 0:
        return {
            'peak_count': 0,
            'peak_rate': 0,
            'mean_peak_prominence': 0,
            'max_peak_prominence': 0,
            'peak_position_mean': 0.5,
            'peak_position_std': 0,
            'peak_width_mean': 0
        }
    
    # Normalize feature sequence to [0,1]
    if np.max(feature_sequence) > np.min(feature_sequence):
        normalized = (feature_sequence - np.min(feature_sequence)) / (np.max(feature_sequence) - np.min(feature_sequence))
    else:
        normalized = np.zeros_like(feature_sequence)
    
    # Find peaks with try/except to handle potential errors
    try:
        peaks, properties = scipy.signal.find_peaks(
            normalized,
            distance=distance,
            prominence=prominence,
            width=width
        )
    except Exception as e:
        print(f"Error finding peaks: {e}")
        return {
            'peak_count': 0,
            'peak_rate': 0,
            'mean_peak_prominence': 0,
            'max_peak_prominence': 0,
            'peak_position_mean': 0.5,
            'peak_position_std': 0,
            'peak_width_mean': 0
        }
    
    # Compute peak features
    peak_count = len(peaks)
    peak_rate = peak_count / n_frames
    
    if peak_count > 0:
        # Make sure properties contains all expected keys
        peak_prominences = properties.get('prominences', np.zeros(peak_count))
        peak_widths = properties.get('widths', np.zeros(peak_count))
        peak_positions = peaks / n_frames  # Normalize positions to [0,1]
        
        mean_peak_prominence = np.mean(peak_prominences)
        max_peak_prominence = np.max(peak_prominences) if len(peak_prominences) > 0 else 0
        peak_position_mean = np.mean(peak_positions)
        peak_position_std = np.std(peak_positions)
        peak_width_mean = np.mean(peak_widths) / n_frames if len(peak_widths) > 0 else 0  # Normalize to utterance length
    else:
        mean_peak_prominence = 0
        max_peak_prominence = 0
        peak_position_mean = 0.5  # Default to middle
        peak_position_std = 0
        peak_width_mean = 0
    
    return {
        'peak_count': peak_count,
        'peak_rate': peak_rate,
        'mean_peak_prominence': mean_peak_prominence,
        'max_peak_prominence': max_peak_prominence,
        'peak_position_mean': peak_position_mean,
        'peak_position_std': peak_position_std,
        'peak_width_mean': peak_width_mean
    }

def extract_energy_envelope(audio_signal, sample_rate=16000, window_size=0.025, window_step=0.010):
    """
    Extract energy envelope from audio signal.
    
    Args:
        audio_signal: Raw audio samples
        sample_rate: Sample rate in Hz
        window_size: Window size in seconds
        window_step: Window step in seconds
        
    Returns:
        Energy envelope (one value per frame)
    """
    audio_signal = np.array(audio_signal) 
    
    # Handle empty array
    if len(audio_signal) == 0:
        return np.array([])
    
    # Ensure audio is 1D
    if len(audio_signal.shape) > 1:
        audio_signal = audio_signal.flatten()
    
    # Convert window sizes from seconds to samples
    window_length = int(window_size * sample_rate)
    hop_length = int(window_step * sample_rate)
    
    # Ensure valid window and hop lengths
    window_length = max(window_length, 1)
    hop_length = max(hop_length, 1)
    
    try:
        # Compute RMS energy in each frame
        energy = librosa.feature.rms(
            y=audio_signal,
            frame_length=window_length,
            hop_length=hop_length
        )[0]
    except Exception as e:
        print(f"Error extracting energy envelope: {e}")
        return np.array([])
    
    return energy

def compute_envelope_features(envelope):
    """
    Compute features from energy envelope.
    
    Args:
        envelope: Energy envelope (one value per frame)
        
    Returns:
        Dictionary of envelope features
    """
    envelope = np.array(envelope)  
    
    # Handle empty array
    if len(envelope) < 2:
        return {
            'mean_attack_rate': 0,
            'mean_decay_rate': 0,
            'attack_decay_ratio': 0,
            'envelope_modulation': 0,
            'attack_count': 0,
            'decay_count': 0
        }
    
    # Normalize envelope
    if np.max(envelope) > np.min(envelope):
        normalized = (envelope - np.min(envelope)) / (np.max(envelope) - np.min(envelope))
    else:
        normalized = np.zeros_like(envelope)
    
    # Compute attack and decay
    attack_rates = []
    decay_rates = []
    
    # Find regions of increasing energy (attacks) and decreasing energy (decays)
    in_attack = False
    in_decay = False
    attack_start = 0
    decay_start = 0
    
    for i in range(1, len(normalized)):
        # Check for attack (significant increase)
        if normalized[i] > normalized[i-1] + 0.05:
            if not in_attack:
                in_attack = True
                attack_start = i - 1
            in_decay = False
        # Check for decay (significant decrease)
        elif normalized[i] < normalized[i-1] - 0.05:
            if in_attack:
                # End of attack, calculate rate
                attack_duration = i - attack_start
                attack_magnitude = normalized[i-1] - normalized[attack_start]
                if attack_duration > 0:
                    attack_rates.append(attack_magnitude / attack_duration)
                in_attack = False
            
            if not in_decay:
                in_decay = True
                decay_start = i - 1
        # Check for end of decay
        elif in_decay and (normalized[i] >= normalized[i-1] or i == len(normalized) - 1):
            # End of decay, calculate rate
            decay_duration = i - decay_start
            decay_magnitude = normalized[decay_start] - normalized[i-1]
            if decay_duration > 0:
                decay_rates.append(decay_magnitude / decay_duration)
            in_decay = False
    
    # Compute summary features
    mean_attack_rate = np.mean(attack_rates) if attack_rates else 0
    mean_decay_rate = np.mean(decay_rates) if decay_rates else 0
    attack_decay_ratio = mean_attack_rate / mean_decay_rate if mean_decay_rate > 0 else 0
    
    # Compute envelope modulation
    modulation = np.std(np.diff(normalized)) if len(normalized) > 1 else 0
    
    return {
        'mean_attack_rate': mean_attack_rate,
        'mean_decay_rate': mean_decay_rate,
        'attack_decay_ratio': attack_decay_ratio,
        'envelope_modulation': modulation,
        'attack_count': len(attack_rates),
        'decay_count': len(decay_rates)
    }

def extract_complete_trajectory_features(utterance, sample_rate=16000):
    """
    Extract comprehensive trajectory-based features from a speech utterance.
    
    Args:
        utterance: Audio samples
        sample_rate: Sample rate in Hz
        
    Returns:
        Dictionary of trajectory features
    """
    utterance = np.array(utterance)  
    
    # Handle empty utterance
    if len(utterance) == 0:
        return {'error': 'Empty utterance'}
    
    features = {}
    
    try:
        # Extract energy envelope
        energy_envelope = extract_energy_envelope(utterance, sample_rate)
        
        # Skip if energy envelope extraction failed
        if len(energy_envelope) == 0:
            return {'error': 'Failed to extract energy envelope'}
        
        # Extract F0 contour (pitch) with error handling
        try:
            f0, voiced_flag, _ = librosa.pyin(
                y=utterance,
                fmin=librosa.note_to_hz('C2'),
                fmax=librosa.note_to_hz('C7'),
                sr=sample_rate,
                frame_length=2048,
                hop_length=512
            )
            # Replace NaN values with zeros for unvoiced frames
            f0 = np.nan_to_num(f0)
        except Exception as e:
            print(f"Error extracting pitch: {e}")
            f0 = np.zeros(len(energy_envelope))
        
        # 1. Linear trends
        features['energy_trend'] = compute_linear_trend(energy_envelope)
        features['f0_trend'] = compute_linear_trend(f0[f0 > 0]) if np.any(f0 > 0) else 0
        
        # 2. Dynamism features
        features['energy_dynamism'] = compute_feature_dynamism(energy_envelope)
        features['f0_dynamism'] = compute_feature_dynamism(f0[f0 > 0]) if np.any(f0 > 0) else 0
        
        # 3. Burst features
        energy_burst_features = compute_burst_features(energy_envelope)
        features.update({f'energy_{k}': v for k, v in energy_burst_features.items()})
        
        # 4. Peak analysis
        energy_peak_features = analyze_peaks(energy_envelope)
        features.update({f'energy_{k}': v for k, v in energy_peak_features.items()})
        
        # 5. Envelope features
        envelope_features = compute_envelope_features(energy_envelope)
        features.update(envelope_features)
        
        # # 6. Combined joy-specific features
        # # Joy often has quick attacks, multiple peaks, and rising contours
        # features['joy_signature'] = (
        #     0.3 * features['energy_trend'] +
        #     0.2 * features.get('energy_burst_rate', 0) +
        #     0.2 * features['mean_attack_rate'] +
        #     0.15 * features.get('energy_peak_rate', 0) +
        #     0.15 * features['energy_dynamism']
        # )
        
    except Exception as e:
        print(f"Error extracting trajectory features: {e}")
        return {'error': str(e)}
    
    return features

In [6]:
import os
# Define the expected features
feature_map = {
    'energy_trend',
    'f0_trend',
    'energy_dynamism',
    'f0_dynamism',
    'energy_burst_count',
    'energy_burst_rate',
    'energy_mean_burst_magnitude',
    'energy_max_burst_magnitude',
    'energy_burst_coverage',
    'energy_first_burst_position',
    'energy_peak_count',
    'energy_peak_rate',
    'energy_mean_peak_prominence',
    'energy_max_peak_prominence',
    'energy_peak_position_mean',
    'energy_peak_position_std',
    'energy_peak_width_mean',
    'mean_attack_rate',
    'mean_decay_rate',
    'attack_decay_ratio',
    'envelope_modulation',
    'attack_count',
    'decay_count',
    'joy_signature'
}
features_df = pd.DataFrame()
# Process each audio file and extract features
for idx, row in data_df.iterrows():
    audio_path = os.path.join("enhanced_audio/", row['Video_ID'])
    try:
        audio, sr = librosa.load(audio_path, sr=16000)
        features = extract_complete_trajectory_features(audio, sample_rate=16000)
        # Convert features dict to DataFrame row and concatenate
        features_row = pd.DataFrame([features])
        features_df = pd.concat([features_df, features_row], ignore_index=True)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        # Add empty features for failed extraction
        empty_features = {feature: np.nan for feature in feature_map}
        empty_df = pd.DataFrame([empty_features])
        features_df = pd.concat([features_df, empty_df], ignore_index=True)

In [7]:
features_df

,energy_trend,f0_trend,energy_dynamism,f0_dynamism,energy_burst_count,energy_burst_rate,energy_mean_burst_magnitude,energy_max_burst_magnitude,energy_burst_coverage,energy_first_burst_position,...,energy_max_peak_prominence,energy_peak_position_mean,energy_peak_position_std,energy_peak_width_mean,mean_attack_rate,mean_decay_rate,attack_decay_ratio,envelope_modulation,attack_count,decay_count
0,-0.078078,0.000000,0.050089,0.000000,4,0.021739,2.207174,2.748440,0.114130,0.288043,...,0.999919,0.414130,0.178916,0.0,0.101344,0.067336,1.505051,0.068234,4,6
1,0.356541,0.000000,0.078701,0.000000,2,0.024390,2.058870,2.083886,0.134146,0.512195,...,0.999860,0.606707,0.161643,0.0,0.097599,0.048729,2.002889,0.103835,4,2
2,0.006085,-0.229032,0.058266,0.099645,1,0.008197,1.794374,1.794374,0.024590,0.737705,...,0.892604,0.689891,0.290509,0.0,0.046739,0.051792,0.902443,0.071476,7,8
3,-0.041621,-0.800000,0.187279,0.358173,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.770479,0.461538,0.000000,0.0,0.058002,0.000000,0.000000,0.213741,2,0
4,0.087883,-0.379507,0.050433,0.049964,6,0.007435,2.097808,2.556380,0.068154,0.071871,...,0.986242,0.513253,0.283036,0.0,0.041151,0.048238,0.853096,0.061954,38,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,-0.394373,-1.034859,0.046598,0.063598,4,0.015326,2.065801,2.348283,0.134100,0.026820,...,0.999905,0.352107,0.224787,0.0,0.060479,0.057078,1.059588,0.063761,7,7
1739,-0.264207,-0.748741,0.034949,0.122124,3,0.009677,2.091850,2.661713,0.103226,0.064516,...,0.999474,0.450896,0.260050,0.0,0.034715,0.066308,0.523543,0.046427,8,6
1740,0.518551,-0.724099,0.061655,0.082897,9,0.014851,1.971561,2.149727,0.061056,0.500000,...,0.999696,0.583828,0.253028,0.0,0.062074,0.057972,1.070771,0.079818,30,26
1741,0.600386,0.495345,0.054880,0.191285,2,0.015504,1.971869,2.003880,0.131783,0.519380,...,0.992537,0.790698,0.189037,0.0,0.070321,0.057777,1.217118,0.076048,4,5


In [8]:
joy_data_df = pd.concat([data_df, features_df], axis=1)
joy_data_df.to_csv("joy_data_df.csv", index=False)

In [9]:
joy_data_df

,Emotion,Video_ID,Gender,energy_trend,f0_trend,energy_dynamism,f0_dynamism,energy_burst_count,energy_burst_rate,energy_mean_burst_magnitude,...,energy_max_peak_prominence,energy_peak_position_mean,energy_peak_position_std,energy_peak_width_mean,mean_attack_rate,mean_decay_rate,attack_decay_ratio,envelope_modulation,attack_count,decay_count
0,joy,dia2_utt2_enhanced.wav,male,-0.078078,0.000000,0.050089,0.000000,4,0.021739,2.207174,...,0.999919,0.414130,0.178916,0.0,0.101344,0.067336,1.505051,0.068234,4,6
1,joy,dia2_utt10_enhanced.wav,male,0.356541,0.000000,0.078701,0.000000,2,0.024390,2.058870,...,0.999860,0.606707,0.161643,0.0,0.097599,0.048729,2.002889,0.103835,4,2
2,joy,dia2_utt12_enhanced.wav,male,0.006085,-0.229032,0.058266,0.099645,1,0.008197,1.794374,...,0.892604,0.689891,0.290509,0.0,0.046739,0.051792,0.902443,0.071476,7,8
3,joy,dia4_utt1_enhanced.wav,female,-0.041621,-0.800000,0.187279,0.358173,0,0.000000,0.000000,...,0.770479,0.461538,0.000000,0.0,0.058002,0.000000,0.000000,0.213741,2,0
4,joy,dia4_utt8_enhanced.wav,female,0.087883,-0.379507,0.050433,0.049964,6,0.007435,2.097808,...,0.986242,0.513253,0.283036,0.0,0.041151,0.048238,0.853096,0.061954,38,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1738,joy,dia1034_utt1_enhanced.wav,female,-0.394373,-1.034859,0.046598,0.063598,4,0.015326,2.065801,...,0.999905,0.352107,0.224787,0.0,0.060479,0.057078,1.059588,0.063761,7,7
1739,joy,dia1035_utt6_enhanced.wav,male,-0.264207,-0.748741,0.034949,0.122124,3,0.009677,2.091850,...,0.999474,0.450896,0.260050,0.0,0.034715,0.066308,0.523543,0.046427,8,6
1740,joy,dia1036_utt0_enhanced.wav,male,0.518551,-0.724099,0.061655,0.082897,9,0.014851,1.971561,...,0.999696,0.583828,0.253028,0.0,0.062074,0.057972,1.070771,0.079818,30,26
1741,joy,dia1038_utt1_enhanced.wav,female,0.600386,0.495345,0.054880,0.191285,2,0.015504,1.971869,...,0.992537,0.790698,0.189037,0.0,0.070321,0.057777,1.217118,0.076048,4,5


In [10]:
joy_data_df_m = joy_data_df[joy_data_df['Gender']=="male"]
joy_data_df_f = joy_data_df[joy_data_df['Gender']=="female"]
joy_data_df_m.to_csv("joy_data_df_m.csv", index=False)
joy_data_df_f.to_csv("joy_data_df_f.csv", index=False)
joy_data_df_m
joy_data_df_f

,Emotion,Video_ID,Gender,energy_trend,f0_trend,energy_dynamism,f0_dynamism,energy_burst_count,energy_burst_rate,energy_mean_burst_magnitude,...,energy_max_peak_prominence,energy_peak_position_mean,energy_peak_position_std,energy_peak_width_mean,mean_attack_rate,mean_decay_rate,attack_decay_ratio,envelope_modulation,attack_count,decay_count
3,joy,dia4_utt1_enhanced.wav,female,-0.041621,-0.800000,0.187279,0.358173,0,0.000000,0.000000,...,0.770479,0.461538,0.000000,0.0,0.058002,0.000000,0.000000,0.213741,2,0
4,joy,dia4_utt8_enhanced.wav,female,0.087883,-0.379507,0.050433,0.049964,6,0.007435,2.097808,...,0.986242,0.513253,0.283036,0.0,0.041151,0.048238,0.853096,0.061954,38,40
6,joy,dia4_utt10_enhanced.wav,female,-0.329277,0.966838,0.035097,0.175716,1,0.006494,3.167485,...,0.997513,0.214286,0.109559,0.0,0.034373,0.049011,0.701322,0.049678,2,2
17,joy,dia11_utt8_enhanced.wav,female,-0.385218,-0.204629,0.049844,0.149197,4,0.022472,2.258409,...,0.999857,0.253745,0.158483,0.0,0.071077,0.067013,1.060639,0.067593,5,7
18,joy,dia11_utt9_enhanced.wav,female,0.106582,0.149889,0.027911,0.069143,5,0.007429,2.914022,...,0.999613,0.653343,0.218516,0.0,0.054635,0.054720,0.998450,0.041084,8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,joy,dia1029_utt6_enhanced.wav,female,-0.290193,-0.747553,0.046325,0.115404,3,0.016304,2.230014,...,0.999945,0.454710,0.256294,0.0,0.060094,0.045630,1.316976,0.062726,6,6
1732,joy,dia1029_utt7_enhanced.wav,female,0.176437,-0.196220,0.065349,0.115112,3,0.019231,1.922154,...,0.999044,0.620192,0.240913,0.0,0.055729,0.066122,0.842827,0.080853,9,8
1733,joy,dia1029_utt14_enhanced.wav,female,0.051275,-0.326936,0.072577,0.183639,3,0.011152,1.972174,...,0.997653,0.553903,0.224961,0.0,0.078351,0.052242,1.499783,0.091003,14,13
1738,joy,dia1034_utt1_enhanced.wav,female,-0.394373,-1.034859,0.046598,0.063598,4,0.015326,2.065801,...,0.999905,0.352107,0.224787,0.0,0.060479,0.057078,1.059588,0.063761,7,7


In [11]:
joy_data_df_m

,Emotion,Video_ID,Gender,energy_trend,f0_trend,energy_dynamism,f0_dynamism,energy_burst_count,energy_burst_rate,energy_mean_burst_magnitude,...,energy_max_peak_prominence,energy_peak_position_mean,energy_peak_position_std,energy_peak_width_mean,mean_attack_rate,mean_decay_rate,attack_decay_ratio,envelope_modulation,attack_count,decay_count
0,joy,dia2_utt2_enhanced.wav,male,-0.078078,0.000000,0.050089,0.000000,4,0.021739,2.207174,...,0.999919,0.414130,0.178916,0.0,0.101344,0.067336,1.505051,0.068234,4,6
1,joy,dia2_utt10_enhanced.wav,male,0.356541,0.000000,0.078701,0.000000,2,0.024390,2.058870,...,0.999860,0.606707,0.161643,0.0,0.097599,0.048729,2.002889,0.103835,4,2
2,joy,dia2_utt12_enhanced.wav,male,0.006085,-0.229032,0.058266,0.099645,1,0.008197,1.794374,...,0.892604,0.689891,0.290509,0.0,0.046739,0.051792,0.902443,0.071476,7,8
5,joy,dia4_utt9_enhanced.wav,male,0.167485,-0.691310,0.068863,0.114841,1,0.007299,2.038836,...,0.966931,0.512165,0.230647,0.0,0.063601,0.045218,1.406547,0.082006,8,11
7,joy,dia6_utt0_enhanced.wav,male,-0.062900,0.000000,0.062873,0.000000,5,0.015480,1.878767,...,0.998252,0.497764,0.243772,0.0,0.059133,0.061680,0.958708,0.079231,19,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1736,joy,dia1032_utt7_enhanced.wav,male,-0.055923,-0.293728,0.037549,0.072827,5,0.007800,2.037465,...,0.991232,0.517534,0.346723,0.0,0.036241,0.043680,0.829703,0.049002,19,19
1737,joy,dia1033_utt2_enhanced.wav,male,0.060887,-0.167961,0.048635,0.112263,7,0.015453,1.929941,...,0.999799,0.495953,0.302296,0.0,0.054989,0.048083,1.143633,0.063235,14,20
1739,joy,dia1035_utt6_enhanced.wav,male,-0.264207,-0.748741,0.034949,0.122124,3,0.009677,2.091850,...,0.999474,0.450896,0.260050,0.0,0.034715,0.066308,0.523543,0.046427,8,6
1740,joy,dia1036_utt0_enhanced.wav,male,0.518551,-0.724099,0.061655,0.082897,9,0.014851,1.971561,...,0.999696,0.583828,0.253028,0.0,0.062074,0.057972,1.070771,0.079818,30,26
